In [37]:
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import re
import unidecode
import nltk
from nltk.corpus import stopwords
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore

In [38]:
dates = tuple((f'01/01/{year}', f'31/12/{year}') for year in range(2000, 2027))
dates = (('10/10/2000', dates[0][1]),) + dates[1:]
scraped_dates = dates[:10]
df = pd.concat([pd.read_csv(f'{date[0][-4:]}.csv', encoding='utf-8') for date in scraped_dates])
df['Data'] = df['Data'].str.split('-').str[::-1].str.join('/')
df = df[df['Hora'] != '*']
df['Data'] = pd.to_datetime(df['Data'] + ' ' + df['Hora'], format='%d/%m/%Y %Hh%M')
df = df.set_index('Data')
df.drop(['Hora'], axis=1, inplace=True)
df = df[df['Orador'].str.contains(",")]
df['Partido'] = df['Orador'].str.split(',').str[1]
df['Orador'] = df['Orador'].str.split(',').str[0]
df['Estado'] = df['Partido'].str.split('-').str[1]
df['Partido'] = df['Partido'].str.split('-').str[0]
df['Partido'] = df['Partido'].str.strip()
df['Discurso_Original'] = df['Discurso']
#drop partido longer than 6 characters
df = df[df['Partido'].str.len() < 7]

In [ ]:
df['Orador'] = df['Orador'].apply(lambda x: re.sub(r"\(.*\)", "", x) if isinstance(x, str) else x)

remove = [
    "Sr",
    "Sra",
    "Srª",
    "Srº",
    "Presidente",
    "Presidenta",
    "Sras",
    "Sres",
    "Srs",
    "Exmo",
    "Exma",
    "V Exa",
    "Deputados",
    "S Exa",
    "Exa",
    "º",
    "ª",
]

remove_pattern = r'\b(?:' + '|'.join(map(re.escape, remove)) + r')\b'
df['Discurso'] = df['Discurso'].str.replace(remove_pattern, '', regex=True)

patterns = [r"\([^)]*\)|[A-Z]{2,}|[^\w\s]|\d+", r".*–", r"\s{2,}"]
for pattern in patterns:
    df['Discurso'] = df['Discurso'].apply(lambda x: re.sub(pattern, ' ', x) if isinstance(x, str) else x)

for i in range(5):
    print(df['Discurso'][i])

In [ ]:
# lowercasing
df['Discurso'] = df['Discurso'].apply(lambda x: x.lower() if isinstance(x, str) else x)
# remove accents
df['Discurso'] = df['Discurso'].apply(lambda x: unidecode.unidecode(x) if isinstance(x, str) else x)
for i in range(5):
    print(df['Discurso'][i])

o deputado severino cavalcanti ministro paulo costa leite do superior tribunal de justica ilmo jose pinto mota filho representante do governo do estado do rio grande do sul ilmo jornalista fabio marcal representante do correio do povo e especialmente o recem eleito prefeito de uruguaiana deputado caio riela e sua esposa senhoras e senhores convidados prezados colaboradores funcionarios desta casa a sessao solene de hoje foi originalmente requerida pelo deputado caio riela e com a licenca de s em funcao da candidatura a prefeitura do municipio de uruguaiana coube a nos esta honrosa distincao de homenagear dois jornais de grande importancia para o rio grande do sul e para o brasil o dia o de outubro festejou o o aniversario do correio do povo e o o de a gazeta de alegrete a gazeta de alegrete fundada em o de outubro de e o mais antigo jornal em circulacao no estado do rio grande do sul e o oitavo em antiguidade no brasil e tambem uma das cinco publicacoes mais antigas do mundo editadas e

/var/folders/xd/g9cbpcws3m14495hf_1bxjyh0000gn/T/ipykernel_58752/3367473474.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df['Discurso'][i])


In [ ]:
nltk.download('stopwords')
stop = stopwords.words('portuguese')
df['Discurso'] = df['Discurso'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop) and len(word) > 2]) if isinstance(x, str) else x)
for i in range(5):
    print(df['Discurso'][i])

[nltk_data] Downloading package stopwords to /Users/r2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


deputado severino cavalcanti ministro paulo costa leite superior tribunal justica ilmo jose pinto mota filho representante governo estado rio grande sul ilmo jornalista fabio marcal representante correio povo especialmente recem eleito prefeito uruguaiana deputado caio riela esposa senhoras senhores convidados prezados colaboradores funcionarios desta casa sessao solene hoje originalmente requerida deputado caio riela licenca funcao candidatura prefeitura municipio uruguaiana coube honrosa distincao homenagear dois jornais grande importancia rio grande sul brasil dia outubro festejou aniversario correio povo gazeta alegrete gazeta alegrete fundada outubro antigo jornal circulacao estado rio grande sul oitavo antiguidade brasil tambem cinco publicacoes antigas mundo editadas lingua portuguesa fundador luiz freitas valle barao ibirocay primeiro editor engenheiro militar jesuino melchiades souza familia prunes jose celestino teodonio cirino jose fredolino lourenco sobrinho mario lourenco 

/var/folders/xd/g9cbpcws3m14495hf_1bxjyh0000gn/T/ipykernel_58752/1160792643.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df['Discurso'][i])


In [ ]:
# # remove people names using Spacy NER in portuguese
# import spacy
# import pt_core_news_md
# nlp = pt_core_news_md.load()
# df['Discurso'] = df['Discurso'].apply(lambda x: ' '.join([word for word in x.split() if not nlp(word)[0].ent_type_]) if isinstance(x, str) else x)


In [ ]:
# nltk.download('rslp')
# stemmer = nltk.stem.RSLPStemmer()
# df['Discurso'] = df['Discurso'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]) if isinstance(x, str) else x)
# for i in range(5):
#     print(df['Discurso'][i])

In [ ]:
# dropna on "Discurso"
df = df.dropna(subset=['Discurso'])

In [ ]:
# vocabulary: 10000 most frequent words
vectorizer = CountVectorizer(max_features=15000)
dtm = vectorizer.fit_transform(df["Discurso"])
lda = LatentDirichletAllocation(
    n_components=300,
    random_state=0,
    max_iter=10,
    verbose=1,
)
lda.fit(dtm)

iteration: 1 of max_iter: 50
iteration: 2 of max_iter: 50
iteration: 3 of max_iter: 50
iteration: 4 of max_iter: 50
iteration: 5 of max_iter: 50
iteration: 6 of max_iter: 50
iteration: 7 of max_iter: 50
iteration: 8 of max_iter: 50
iteration: 9 of max_iter: 50
iteration: 10 of max_iter: 50
iteration: 11 of max_iter: 50
iteration: 12 of max_iter: 50
iteration: 13 of max_iter: 50
iteration: 14 of max_iter: 50
iteration: 15 of max_iter: 50
iteration: 16 of max_iter: 50
iteration: 17 of max_iter: 50
iteration: 18 of max_iter: 50
iteration: 19 of max_iter: 50
iteration: 20 of max_iter: 50
iteration: 21 of max_iter: 50
iteration: 22 of max_iter: 50
iteration: 23 of max_iter: 50
iteration: 24 of max_iter: 50
iteration: 25 of max_iter: 50
iteration: 26 of max_iter: 50
iteration: 27 of max_iter: 50
iteration: 28 of max_iter: 50
iteration: 29 of max_iter: 50
iteration: 30 of max_iter: 50
iteration: 31 of max_iter: 50
iteration: 32 of max_iter: 50
iteration: 33 of max_iter: 50
iteration: 34 of ma

LatentDirichletAllocation(max_iter=50, n_components=300, random_state=0,
                          verbose=1)

In [ ]:
#show results

for index, topic in enumerate(lda.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    # print original word instead of lemmatized
    
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['acordo', 'vai', 'regional', 'todos', 'todas', 'sao', 'pais', 'sede', 'cidade', 'capital', 'nao', 'estado', 'brasilia', 'janeiro', 'rio']


THE TOP 15 WORDS FOR TOPIC #1
['uniao', 'nacional', 'mendes', 'congresso', 'recursos', 'governo', 'paulo', 'contas', 'sao', 'tribunal', 'irregularidades', 'nao', 'milhoes', 'obras', 'obra']


THE TOP 15 WORDS FOR TOPIC #2
['primeira', 'valor', 'acordo', 'funcao', 'avaliacao', 'conselho', 'mil', 'administracao', 'distribuicao', 'creditos', 'diretores', 'nao', 'apos', 'reais', 'fazenda']


THE TOP 15 WORDS FOR TOPIC #3
['habitantes', 'bianco', 'saude', 'novo', 'buritis', 'velho', 'ano', 'nao', 'casos', 'malaria', 'governo', 'populacao', 'municipios', 'estado', 'rondonia']


THE TOP 15 WORDS FOR TOPIC #4
['diretora', 'dia', 'tambem', 'tributaria', 'qualidade', 'anos', 'nao', 'pode', 'pais', 'brasil', 'bancada', 'palocci', 'preto', 'cafe', 'ribeirao']


THE TOP 15 WORDS FOR TOPIC #5
['brasileiros', 'registrar', 'tambem', 

In [ ]:
# add topic column to df
topic_results = lda.transform(dtm)
df['Topico'] = topic_results.argmax(axis=1)
df['Topico'] = df['Topico'].apply(lambda x: x + 1)
df.head()

,Sessão,Fase,URL_Discurso,Discurso,Orador,Publicação,Partido,Estado,Discurso_Original,Topico
Data,,,,,,,,,,
2000-10-10 10:24:00,179.2.51.O,HOMENAGEM,https://www.camara.leg.br/internet/sitaqweb/Te...,deputado severino cavalcanti ministro paulo co...,EDIR OLIVEIRA,DCD11/10/2000 PAG. 50369,PTB,RS,O SR. EDIR OLIVEIRA (PTB-RS. Pronuncia o segui...,240
2000-10-10 10:42:00,179.2.51.O,HOMENAGEM,https://www.camara.leg.br/internet/sitaqweb/Te...,satisfacao registramos presencas profa doralia...,SEVERINO CAVALCANTI,DCD11/10/2000 PAG. 50371,PPB,PE,O SR. PRESIDENTE (Severino Cavalcanti) – Com s...,10
2000-10-10 10:56:00,179.2.51.O,HOMENAGEM,https://www.camara.leg.br/internet/sitaqweb/Te...,ministro paulo costa leite jose pinto fabio ma...,SEVERINO CAVALCANTI,DCD11/10/2000 PAG. 50373,PPB,PE,O SR. PRESIDENTE (Severino Cavalcanti) – Exmo....,10
2000-10-10 14:20:00,180.2.51.O,PEQUENO EXPEDIENTE,https://www.camara.leg.br/internet/sitaqweb/Te...,vez ocupo tribuna abordar tema educacao examin...,GILMAR MACHADO,DCD11/10/2000 PAG. 50375,PT,MG,O SR. GILMAR MACHADO (PT-MG. Sem revisão do or...,245
2000-10-10 14:32:00,180.2.51.O,PEQUENO EXPEDIENTE,https://www.camara.leg.br/internet/sitaqweb/Te...,numeros divulgados ultima pesquisa instituto p...,JOSÉ ANTONIO ALMEIDA,DCD11/10/2000 PAG. 50377,PSB,MA,O SR. JOSÉ ANTONIO ALMEIDA (Bloco/PSB-MA. Sem ...,55


In [ ]:
# create copy where Topico is 77
df_77 = df[df['Topico'] == 77].copy()
#describe
df_77.describe()

,Topico
count,5.0
mean,77.0
std,0.0
min,77.0
25%,77.0
50%,77.0
75%,77.0
max,77.0


In [ ]:
#print all speeches one by one
for speech in df_77['Discurso_Original']:
    print(speech)

O SR. CORIOLANO SALES (Bloco/PMDB-BA. Pronuncia o seguinte discurso.) - Sr. Presidente, Sras. e Srs. Deputados, quero parabenizar o Ministro da Comunicações, Sr. Pimenta da Veiga, e o próprio Governo Federal, pela decisão de montar o Banco Postal do Brasil, para funcionar como entidade financeira praticando todas as atividades bancárias.  É uma decisão muito importante para o desenvolvimento das pequenas atividades econômicas no Brasil, notadamente para o acesso do crédito ao pessoal de baixa renda. Outros países têm no crédito popular, via bancos postais, bancos cooperativos e caixas de poupança e de crédito, a grande alavanca para a promoção do emprego, do trabalho e da renda.  O Banco Postal chega ao Brasil com, pelo menos, cem anos de atraso em relação ao Japão, que hoje possui mais de 2 trilhões de dólares de poupança.  Na França, o Banco dos Correios é o "banco do pessoal de baixa renda". De cada dois franceses, um é cliente do Banco dos Correios.  O Banco Postal cumpre, portanto